In [1]:
import keras
from keras.models import Sequential
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape, Merge
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import  BatchNormalization
from keras.layers.convolutional import *
import numpy as np
import tensorflow as tf
# import theano
import  matplotlib.pyplot as plt
from PIL import Image
import matplotlib
import itertools
from numpy import*
from sklearn.utils import shuffle
%matplotlib inline
import random as rn
import os
import math
import copy
os.environ['PYTHONASHSEED']= '0'

Using TensorFlow backend.


In [2]:
######## parameter #####
batch_size = 8
n_epochs = 500

In [3]:
import os
classes_name = []
files_name = []
for root, dirs, files in os.walk('lfw'):  
    for name in dirs:
        classes_name.append(name)
    for name1 in files:
        files_name.append(name1)




In [4]:

print(len(classes_name))
print(len(files_name))

5749
13233


In [5]:

#  print(classes_name[0])

In [6]:
############ Define  function to split  dataset #######################

# import shutil, sys  
# def split_dataset_into_test_and_train_sets(all_data_dir, training_data_dir, testing_data_dir, testing_data_pct):
    
#     if not os.path.exists(training_data_dir):
#         os.makedirs(training_data_dir)
#     if not os.path.exists(testing_data_dir):
#         os.makedirs(testing_data_dir)
#     # Recreate testing and training directories
#     if testing_data_dir.count('/') > 1:
#         shutil.rmtree(testing_data_dir, ignore_errors=False)
#         os.makedirs(testing_data_dir)
#         print("Successfully cleaned directory " + testing_data_dir)
#     else:
#         print("Refusing to delete testing data directory " + testing_data_dir + " as we prevent you from doing stupid things!")

#     if training_data_dir.count('/') > 1:
#         shutil.rmtree(training_data_dir, ignore_errors=False)
#         os.makedirs(training_data_dir)
#         print("Successfully cleaned directory " + training_data_dir)
#     else:
#         print("Refusing to delete testing data directory " + training_data_dir + " as we prevent you from doing stupid things!")

#     num_training_files = 0
#     num_testing_files = 0

#     for subdir, dirs, files in os.walk(all_data_dir):
#         category_name = os.path.basename(subdir)                        ### find  name  of classes

#         # Don't create a subdirectory for the root directory
#         print(category_name + " vs " + os.path.basename(all_data_dir))
#         if category_name == os.path.basename(all_data_dir):
#             continue

#         training_data_category_dir = training_data_dir + '/' + category_name
#         testing_data_category_dir = testing_data_dir + '/' + category_name

#         if not os.path.exists(training_data_category_dir):
#             os.mkdir(training_data_category_dir)

#         if not os.path.exists(testing_data_category_dir):
#             os.mkdir(testing_data_category_dir)

#         for file in files:
#             input_file = os.path.join(subdir, file)
#             if np.random.rand(1) < testing_data_pct:
#                 shutil.copy(input_file, testing_data_dir + '/' + category_name + '/' + file)
#                 num_testing_files += 1
#             else:
#                 shutil.copy(input_file, training_data_dir + '/' + category_name + '/' + file)
#                 num_training_files += 1

#     print("Processed " + str(num_training_files) + " training files.")
#     print("Processed " + str(num_testing_files) + " testing files.")

In [7]:
########### Split data into train_data and test data
#split_dataset_into_test_and_train_sets('lfw','train_data', 'test_data', 0.2)

In [ ]:
# data_generator = ImageDataGenerator(
#                         featurewise_center=False,
#                         featurewise_std_normalization=False,
#                         rotation_range=10,
#                         width_shift_range=0.1,
#                         height_shift_range=0.1,
#                         zoom_range=.1,
#                         horizontal_flip=True)

In [8]:
train_path = 'train_data'
test_path = 'test_data'
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size = (224,224), classes = classes_name, batch_size = batch_size)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size = (224,224), classes = classes_name, batch_size = batch_size)

Found 10541 images belonging to 5749 classes.
Found 2692 images belonging to 5749 classes.


In [9]:
import os
classes_name_train = []
files_name_train = []
classes_name_test = []
files_name_test = []
for root, dirs, files in os.walk('train_data'):  
    for name in dirs:
        classes_name_train.append(name)
    for name1 in files:
        files_name_train.append(name1)
        
for root, dirs, files in os.walk('test_data'):  
    for name in dirs:
        classes_name_test.append(name)
    for name1 in files:
        files_name_test.append(name1)

no_training = len(files_name_train)
no_testing = len(files_name_test)
        
print(len(classes_name_train))
print(len(files_name_train))
print(len(classes_name_test))
print(len(files_name_test))
number_classes = len(classes_name_train)

5749
10541
5749
2692


In [10]:
######### fine_tune vgg_face 
def convblock(cdim, nb, bits=3):
    L = []
    
    for k in range(1,bits+1):
        convname = 'conv'+str(nb)+'_'+str(k)
        #L.append( Convolution2D(cdim, 3, 3, border_mode='same', activation='relu', name=convname) ) # Keras 1
        L.append( Convolution2D(cdim, kernel_size=(3, 3), padding='same', activation='relu', name=convname) ) # Keras 2
    
    L.append( MaxPooling2D((2, 2), strides=(2, 2)) )
    
    return L

In [11]:
def vgg_face_blank():
    
    withDO = True # no effect during evaluation but usefull for fine-tuning
    
    if True:
        mdl = Sequential()
        
        # First layer is a dummy-permutation = Identity to specify input shape
        mdl.add( Permute((1,2,3), input_shape=(224,224,3)) ) # WARNING : 0 is the sample dim

        for l in convblock(64, 1, bits=2):
            mdl.add(l)

        for l in convblock(128, 2, bits=2):
            mdl.add(l)
        
        for l in convblock(256, 3, bits=3):
            mdl.add(l)
            
        for l in convblock(512, 4, bits=3):
            mdl.add(l)
            
        for l in convblock(512, 5, bits=3):
            mdl.add(l)
        
        #mdl.add( Convolution2D(4096, 7, 7, activation='relu', name='fc6') ) # Keras 1
        mdl.add( Convolution2D(4096, kernel_size=(7, 7), activation='relu', name='fc6') ) # Keras 2
        if withDO:
            mdl.add( Dropout(0.5) )
        #mdl.add( Convolution2D(4096, 1, 1, activation='relu', name='fc7') ) # Keras 1
        mdl.add( Convolution2D(4096, kernel_size=(1, 1), activation='relu', name='fc7') ) # Keras 2
        if withDO:
            mdl.add( Dropout(0.5) )
        #mdl.add( Convolution2D(2622, 1, 1, name='fc8') ) # Keras 1
        mdl.add( Convolution2D(number_classes, kernel_size=(1, 1), activation='relu', name='fc8') ) # Keras 2
        mdl.add( Flatten() )
        mdl.add( Activation('softmax') )
        
        return mdl
    
    else:
        # See following link for a version based on Keras functional API :
        # gist.github.com/EncodeTS/6bbe8cb8bebad7a672f0d872561782d9
        raise ValueError('not implemented')

In [12]:
model = vgg_face_blank()

In [13]:

opt = Adam(lr=0.001, decay=10e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# model.compile(loss='hinge', optimizer='adadelta', metrics=['accuracy'])

# sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='hinge', optimizer=sgd, metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
# Save check point
filepath = "weights.VggFace.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose =1, save_best_only = True, mode ='max')
callbacks_list= [checkpoint]


history = model.fit_generator(train_batches, steps_per_epoch = int(no_training / batch_size), validation_data = test_batches, validation_steps = int(no_testing / batch_size), epochs = n_epochs, shuffle=True, callbacks = callbacks_list, verbose =2)

Epoch 1/500
 - 369s - loss: 16.1032 - acc: 5.6948e-04 - val_loss: 16.1181 - val_acc: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to weights.VggFace.best.hdf5
Epoch 2/500
 - 365s - loss: 16.1150 - acc: 1.8983e-04 - val_loss: 16.1181 - val_acc: 0.0000e+00

Epoch 00002: val_acc did not improve
Epoch 3/500
 - 362s - loss: 16.1150 - acc: 1.8983e-04 - val_loss: 16.1181 - val_acc: 0.0000e+00

Epoch 00003: val_acc did not improve
Epoch 4/500
 - 364s - loss: 16.1104 - acc: 4.7456e-04 - val_loss: 16.1181 - val_acc: 0.0000e+00

Epoch 00004: val_acc did not improve
Epoch 5/500


In [ ]:
from keras.models import load_model
model = load_model('weights.VggFace.best.hdf5')

In [ ]:
def pred(kmodel, crpimg, transform=False):
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940
        #
        # WARNING : in this script (https://github.com/rcmalli/keras-vggface) colours are switched
        aux = copy.copy(imarr)
        #imarr[:, :, 0] = aux[:, :, 2]
        #imarr[:, :, 2] = aux[:, :, 0]

        #imarr[:,:,0] -= 129.1863
        #imarr[:,:,1] -= 104.7624
        #imarr[:,:,2] -= 93.5940

    #imarr = imarr.transpose((2,0,1)) # INFO : for 'th' setting of 'dim_ordering'
    imarr = np.expand_dims(imarr, axis=0)

    out = kmodel.predict(imarr)

    best_index = np.argmax(out, axis=1)[0]
    best_name = classes_name[best_index]    ###### 
    
    print(best_index, best_name[0], out[0,best_index], [np.min(out), np.max(out)])

In [ ]:
imgs, labels = next(train_batches)

crpim = imgs # WARNING : we deal with cropping in a latter section, this image is already fit
pred(model, crpim, transform=False)
pred(model, crpim, transform=True)